<a href="https://colab.research.google.com/github/Seongjin1225/AI_SCHOOL_9/blob/main/ML%26DL/12%EC%9B%94%2020%EC%9D%BC/12%EC%9B%94_20%EC%9D%BC_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMDB Review Dataset Loading
from tensorflow.keras.datasets import imdb
(x_data_train, t_data_train), (x_data_test, t_data_test) = \
imdb.load_data(num_words=500)

17464789/17464789 [==============================] - 2s 0us/step


In [ ]:
x_data_train.shape
t_data_train.shape

from tensorflow.keras.preprocessing.sequence import pad_sequences
x_data_train_seq = pad_sequences(x_data_train,maxlen=100)
# 모든 review의 길이가 100개의 token으로 획일화
# 각 review에서 100개가 넘어가는 token은 삭제, 모자라는 token은 padding 처리

# 아래 input_shpape이 100,500인 이유??
# 결국은 one-hot encoding으로 처리해야
# 하나하나의 token이 500개짜리 1차원 ndarray로 표현
# 전체 데이터는 (100,500)

# one-hot enconding 과
from tensorflow.keras.utils import to_categorical
x_data_train_onehot = to_categorical(x_data_train_seq)
x_data_train_onehot.shape

(25000, 100, 500)

In [ ]:
# RNN Model 구현
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from tensorflow.keras.optimizers import Adam

model = Sequential()
model.add(SimpleRNN(units=8,
                    activation='tanh',
                    input_shape=(100,500)))

# output layer
model.add(Dense(units=1,
                activation='sigmoid'))

#

In [ ]:
# 전체 코드 작성
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split  # 여기서는 train과 validation 분리하는 용도로 사용!

from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# 1. Data Loading
(x_data_train, t_data_train), (x_data_test, t_data_test) = \
imdb.load_data(num_words=500)

# 2. 데이터 확인
x_data_train.shape

# 3. data 분리
x_data_train, x_data_valid, t_data_train, t_data_valid = \
train_test_split(x_data_train,
                 t_data_train,
                 test_size=0.2,
                 random_state=42,
                 stratify=t_data_train)

# review 길이 통일
x_data_train_seq = pad_sequences(x_data_train, maxlen=100)
x_data_valid_seq = pad_sequences(x_data_valid, maxlen=100)

# x_data one-hot 형태로 변형해야 입력으로 사용 가능
x_data_train_onehot = to_categorical(x_data_train_seq)
x_data_valid_onehot = to_categorical(x_data_valid_seq)

In [ ]:
# Model 구현
model = Sequential()

model.add(SimpleRNN(units=8,
                    activation='tanh',
                    input_shape=(100,500)))

model.add(Dense(units=1,
                activation='sigmoid'))

model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['acc'])

es_cb = EarlyStopping(monitor='val_loss',
                      patience=4,
                      verbose=1,
                      restore_best_weights=True)

cp_cb = ModelCheckpoint(filepath='./best-simplernn-model.ckpt',
                        monitor='val_loss',
                        verbose=1,
                        save_best_only=True)

history = model.fit(x_data_train_onehot,
                    t_data_train,
                    epochs=100,
                    batch_size=64,
                    validation_data = (x_data_valid_onehot,t_data_valid),
                    callbacks=[es_cb, cp_cb])

In [ ]:
train_loss = history.history['loss']
valid_loss = history.history['val_loss']

plt.plot(train_loss, color='r')
plt.plot(valid_loss, color='b')

In [ ]:
# one-hot encoding 방식이 아니라 word embedding 방식으로
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split  # 여기서는 train과 validation 분리하는 용도로 사용!

from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# 1. Data Loading
(x_data_train, t_data_train), (x_data_test, t_data_test) = \
imdb.load_data(num_words=500)

# 2. 데이터 확인
x_data_train.shape

# 3. data 분리
x_data_train, x_data_valid, t_data_train, t_data_valid = \
train_test_split(x_data_train,
                 t_data_train,
                 test_size=0.2,
                 random_state=42,
                 stratify=t_data_train)

# review 길이 통일
x_data_train_seq = pad_sequences(x_data_train, maxlen=100)
x_data_valid_seq = pad_sequences(x_data_valid, maxlen=100)

# Model 구현
model = Sequential()

model.add(Embedding(input_dim=500,  # vocabulary 수
                    output_dim=16,  # embedding size - 칸의 개수(one-hot의 경우 500개였음)
                    input_length=100))  # time-stamp(길이?)

model.add(SimpleRNN(units=8,
                    activation='tanh'))

model.add(Dense(units=1,
                activation='sigmoid'))

model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['acc'])

es_cb = EarlyStopping(monitor='val_loss',
                      patience=4,
                      verbose=1,
                      restore_best_weights=True)

cp_cb = ModelCheckpoint(filepath='./best-simplernn-model.ckpt',
                        monitor='val_loss',
                        verbose=1,
                        save_best_only=True)

history = model.fit(x_data_train_seq,
                    t_data_train,
                    epochs=100,
                    batch_size=64,
                    validation_data = (x_data_valid_seq,t_data_valid),
                    callbacks=[es_cb, cp_cb])

Epoch 1/100
313/313 [==============================] - ETA: 0s - loss: 0.6675 - acc: 0.5915
Epoch 1: val_loss improved from inf to 0.62218, saving model to ./best-simplernn-model.ckpt
313/313 [==============================] - 42s 122ms/step - loss: 0.6675 - acc: 0.5915 - val_loss: 0.6222 - val_acc: 0.6808
Epoch 2/100
313/313 [==============================] - ETA: 0s - loss: 0.5955 - acc: 0.7106
Epoch 2: val_loss improved from 0.62218 to 0.58054, saving model to ./best-simplernn-model.ckpt
313/313 [==============================] - 31s 99ms/step - loss: 0.5955 - acc: 0.7106 - val_loss: 0.5805 - val_acc: 0.7240
Epoch 3/100
313/313 [==============================] - ETA: 0s - loss: 0.5600 - acc: 0.7485
Epoch 3: val_loss improved from 0.58054 to 0.55860, saving model to ./best-simplernn-model.ckpt
313/313 [==============================] - 31s 99ms/step - loss: 0.5600 - acc: 0.7485 - val_loss: 0.5586 - val_acc: 0.7446
Epoch 4/100
313/313 [==============================] - ETA: 0s - loss:

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
sentences = [
    '강감찬은 나를 너무 너무 좋아해',
    '강감찬은 영화를 좋아해'
]
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)

# 단어 인덱스를 확인해 보아요!
print(f'단어 인덱스는 : {tokenizer.word_index}')

# 숫자로 변경하려면 어떻게 해야 하나요?
word_encoding = tokenizer.texts_to_sequences(sentences)
word_encoding

단어 인덱스는 : {'강감찬은': 1, '너무': 2, '좋아해': 3, '나를': 4, '영화를': 5}


[[1, 4, 2, 2, 3], [1, 5, 3]]

In [ ]:
# 단어 사전에 없는 새로운 단어가 등장하면?
new_sentence = [
    '강감찬은 신사임당을 너무 좋아해'
]
new_word_encoding = tokenizer.texts_to_sequences(new_sentence)
new_word_encoding

# 기본적인 동작은 이렇게 단어사전에 없는 데이터는 변환이 안되기 때문에
# 그냥 없애버려요!

[[1, 2, 3]]

In [ ]:
# 이렇게 처리하면 곤란해요
# OOV(Out Of Vocabulary) token은 따로 처리해줘야 해요
from tensorflow.keras.preprocessing.text import Tokenizer
sentences = [
    '강감찬은 나를 너무 너무 좋아해',
    '강감찬은 영화를 좋아해'
]
tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(sentences)

# 단어 인덱스를 확인해 보아요!
print(f'단어 인덱스는 : {tokenizer.word_index}')

# 숫자로 변경하려면 어떻게 해야 하나요?
word_encoding = tokenizer.texts_to_sequences(sentences)
word_encoding

# # 새로운 단어 처리 ver.
# new_sentence = [
#     '강감찬은 신사임당을 너무 좋아해'
# ]
# new_word_encoding = tokenizer.texts_to_sequences(new_sentence)
# new_word_encoding

# 이제 길이도 맞춰야 해요!
from tensorflow.keras.preprocessing.sequence import pad_sequences
padding = pad_sequences(word_encoding, maxlen=4)
padding

단어 인덱스는 : {'<OOV>': 1, '강감찬은': 2, '너무': 3, '좋아해': 4, '나를': 5, '영화를': 6}


array([[5, 3, 3, 4],
       [0, 2, 6, 4]], dtype=int32)

In [ ]:
# 위처럼 vocabulary를 구성하면 모든 token들이 포함되는 단어사전이 만들어져요
# but, 단어사전이 너무 커진다는 단점 존재
# 일반적으로 빈도수가 높은 단어들만 단어사전에 포함
# 빈도수가 높은 순서로 최대 사전개수 정하고 빈도수가 작은 단어 제외하면 된다

from tensorflow.keras.preprocessing.text import Tokenizer
sentences = [
    '강감찬은 나를 너무 너무 좋아해',
    '강감찬은 영화를 좋아해'
]
tokenizer = Tokenizer(num_words=3,  # 해당 숫자 미만의 값들만 표현!
                      oov_token='<OOV>')
tokenizer.fit_on_texts(sentences)

# 단어 인덱스를 확인해 보아요!
print(f'단어 인덱스는 : {tokenizer.word_index}')

# 숫자로 변경하려면 어떻게 해야 하나요?
word_encoding = tokenizer.texts_to_sequences(sentences)
word_encoding

# 새로운 단어 처리 ver.
new_sentence = [
    '강감찬은 신사임당을 너무 좋아해'
]
new_word_encoding = tokenizer.texts_to_sequences(new_sentence)
new_word_encoding

단어 인덱스는 : {'<OOV>': 1, '강감찬은': 2, '너무': 3, '좋아해': 4, '나를': 5, '영화를': 6}


[[2, 1, 1, 1]]

In [ ]:
# 기본적인 내용을 알아봤으니 이제 실제로 구현 해보아요!
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

# Naver가 제공하는 네이버 영화 리뷰 데이터 로딩
train_file = tf.keras.utils.get_file(
    '/content/ratings_train.txt',  # 파일 저장 경로
    origin = 'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt',  # origin => 데이터 읽어올 경로
    extract=True
)

train_df = pd.read_csv(train_file, sep='\t')  # sep -> 구분자 설정, tab으로 구분되어 있으므로 \t
train_df.head()
train_df.shape

test_file = tf.keras.utils.get_file(
    '/content/ratings_test.txt',  # 파일 저장 경로
    origin = 'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt',  # origin => 데이터 읽어올 경로
    extract=True
)
test_df = pd.read_csv(test_file, sep='\t')
test_df.head()
test_df.shape

(50000, 3)

In [ ]:
# 간단하게 데이터 확인 과정만 해 보아요!
# 긍정과 부정 비율 확인
cnt = train_df['label'].value_counts()
cnt

# 결측치 확인
# 댓글에 5개 결측치 존재 - 추후 삭제 후 진행
train_df.isnull().sum()

id          0
document    5
label       0
dtype: int64

In [ ]:
# 형태소 분석해야
# 다양한 한국어 형태소 분석기 존재
# Kkma, Komoran, Okt, Mecab 등이 있으며
# 어떤 형태소 분석기 사용하느냐에 따라 결과 달라짐
# koNLPy라는 라이브러리 이용해서 사용
# Colab의 경우, 간단히 해당 라이브러리만 설치하면 된다
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 46.6 MB/s eta 0:00:00


In [ ]:
from konlpy.tag import Okt, Komoran, Kkma

kkma = Kkma()
okt = Okt()
komoran = Komoran()

text = '이것은 소리없는 아우성. 저 푸른 해원을 향하여 흔드는 노스텔지어의 손수건'
kkma.morphs(text)  # morphs 함수 사용하면 형태소 추출 가능
okt.morphs(text)
komoran.morphs(text)

In [ ]:
# 데이터 전처리도 진행해야 해요

# 1. 댓글 데이터에는 특수문자, 숫자 같은게 함께 포함되어 있어요
#    정규 표현식 사용해서 영어, 한글, 띄어쓰기만 남기고 나머지는 제거

# 2. 결측치 제거
# 3. stopword(불용어)를 제거
# 4. 단어사전 만들고 숫자로 변경하는 token화 작업 진행
# 5. 동일한 문장 길이로 정리(paading 처리)

#########################################
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

# Naver가 제공하는 네이버 영화 리뷰 데이터 로딩
train_file = tf.keras.utils.get_file(
    '/content/ratings_train.txt',  # 파일 저장 경로
    origin = 'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt',  # origin => 데이터 읽어올 경로
    extract=True
)
train_df = pd.read_csv(train_file, sep='\t')  # sep -> 구분자 설정, tab으로 구분되어 있으므로 \t

test_file = tf.keras.utils.get_file(
    '/content/ratings_test.txt',  # 파일 저장 경로
    origin = 'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt',  # origin => 데이터 읽어올 경로
    extract=True
)
test_df = pd.read_csv(test_file, sep='\t')

# [^A-Za-z가-힣ㄱ-ㅎㅏ-ㅣ ] => ^은 not을 의미(영어, 한글, 공백이 아닌 것 의미)
# 1. 특수문자 제거(정규을 이용!)
train_df['document'] = train_df['document'].str.replace('[^A-Za-z가-힣ㄱ-ㅎㅏ-ㅣ ]','')
train_df.head()

# 2. 결측치 제거(5개)
train_df = train_df.dropna()

# 3. stop word(불용어) 처리
# 관사, 조치사, 조사, 접속사등 의미가 없는 단어들 의미
# 한글에는 이런 불용어가 굉장히 많다 -> 전부 다 처리하기는 불가능하므로
# 자주 사용하는 몇 개만 처리할 예정
from konlpy.tag import Kkma
okt = Okt()
def word_tokenization(text):
    stop_words = ['는', '을', '를', '이', '가', '의', '던', '고', '하', '다',
                  '은', '에', '들', '지', '게', '도']
    return [word for word in okt.morphs(text) if word not in stop_words]

data_train = train_df['document'].apply(lambda x: word_tokenization(x))
data_train.head()

<ipython-input-25-bcce9b3fa27f>:34: FutureWarning: The default value of regex will change from True to False in a future version.
  train_df['document'] = train_df['document'].str.replace('[^A-Za-z가-힣ㄱ-ㅎㅏ-ㅣ ]','')


In [ ]:
# 불용어 처리가 됬다고 가정하고 진행
# 단어 사전을 만들고 문자를 숫자로 변경하는 token화 진행!

from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(data_train)  # 단어사전 생성!

# 일단 총 단어의 개수 확인
print(f'총 단어의 개수는 : {len(tokenizer.word_index)}')

# 단어사전 출력하기
tokenizer.word_index

# 각 토큰이 나타난 수(빈도)를 알아보아요!
tokenizer.word_counts

# 각 토큰이 나타난 수(빈도)에서 숫자만 출력
tokenizer.word_counts.values()

# 단어 빈도수 5회 이상인 단어가 몇개인 지 확인
def get_voca_size(threshold):
    count = 0
    for x in tokenizer.word_counts.values():
        if x > threshold:
            count += 1
    return count

voca_size = get_voca_size(5)

In [ ]:
# 단어사전을 생성
tokenizer = Tokenizer(oov_token='<OOV>',
                      num_words=15000)
tokenizer.fit_on_texts(data_train)

# 이제 각 문장을 숫자 벡터로 변환하여 encoding
data_train_seq = tokenizer.texts_to_sequences(data_train)

# 동일한 길이로 문장 길이를 정리 해야해요
max_length = max([len(x) for x in data_train_seq])

from tensorflow.keras.preprocessing.sequence import pad_sequences
x_data_train = pad_sequences(data_train_seq,
                             maxlen=max_length)

t_data_train = train_df['label'].values

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Bidirectional  # Bidirectional - 양방향 RNN 만들기 위해 사용
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

model = Sequential()

model.add(Embedding(input_dim=15000,
                    output_dim=32,
                    input_length=71))

# 양방향 LSTM을 만들어요!()
model.add(Bidirectional(LSTM(units=16,
                             activation='tanh')))

model.add(Dense(units=32,
                activation='relu'))

model.add(Dense(units=1,
                activation='sigmoid'))

model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['acc'])

# Callback 설정
cp_cb = ModelCheckpoint(filepath='./best_model.ckpt',
                        save_weights_only=True,
                        save_best_only=True,
                        monitor='val_loss',
                        verbose=1)
es_cb = EarlyStopping(monitor='val_loss',
                      patience=3,
                      verbose=1)

# 학습
history = model.fit(x_data_train,
                    t_data_train,
                    epochs=10,
                    batch_size=64,
                    validation_split=0.2,
                    callbacks=[cp_cb, es_cb],
                    verbose=1)


In [ ]:
# 학습이 끝난 다음 prediction을 할 수 있어요
# 예측 데이터는 다음과 같은 데이터 사용
review_sentences = ['내가 만들어도 이것보단 잘 만들겠다.',
                    '너무너무 재미있었습니다. 감사합니다.',
                    '아..내 돈... 돈 아까워 죽을거 같아요',
                    '아나..이것도 영화라고 만들었냐. 무슨 스토리가 산으로 가냐',
                    '감동과 재미가 같이 있는 영화입니다. 훌륭합니다.',
                    '너무너무 재미없다.. 잠와 죽는줄.',
                    '너무너무 재미있다.. 잠이 확깨네.']